In [4]:
import pandas as pd
import pyarrow
from dotenv import load_dotenv
import os
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    log_loss,
    classification_report
)
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from shapely import wkt
from shapely.affinity import translate, scale
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from shapely.affinity import translate, scale
from PIL import Image, ImageDraw

load_dotenv()

file_path = os.getenv('FILE_PATH')

df_sample = pd.read_parquet(file_path + "detailed_woning_type_sample.parquet")
df = pd.read_csv(file_path + "bag_image_summary.csv", dtype="string")
df_joined = pd.merge(df_sample, df, how="left", right_on="bag_id", left_on="bag_nummeraanduidingid")
df = df_joined[df_joined["frontview_exists"].notna()]

# If you want to add the file path to the URLs, set this to True
add_file_path_to_urls = True

# Currently a funda sourced Url goes from: 
# frontview/0797/2000/0002/3888/0797200000023888.jpg
# to: img_dataset/07/079720000002-funda.jpg
def extract_path(url, source):
    if pd.isna(url) or url == '' or url is None:
        return ''
    id = url.rstrip('/').split('/')[-1]
    id, *_ = id.split('.')
    m = re.match(r'(\d{2})', id)
    first_two_digits = m.group(1) if m else ''
    return f"img_dataset/{first_two_digits}/{id}-{source}.jpg"

link_cols = ['frontview_funda_url', 'frontview_google_url', 'frontview_funda_in_business_url']
link_sources = ['funda', 'google', 'funda-in-business'] # Sources are in file name, so need to be added to filename for correct name

for col, source in zip(link_cols, link_sources):
    df[f'{col}_split'] = df[col].map(lambda url: extract_path(url, source))

# If you want to add the file path to the URLs, set add_file_path_to_urls to True
if add_file_path_to_urls:
    df[[f'{col}_split' for col in link_cols]] = df[[f'{col}_split' for col in link_cols]].map(lambda x: file_path + x if x else '')
    add_file_path_to_urls = False

df.to_csv(
    file_path + "Full_preprocessed_detailed_house.csv",
    index=False,
    encoding='utf-8',
)

df = pd.read_csv(file_path + "Full_preprocessed_detailed_house.csv", dtype="string")

# Verschillend is a special case, so we remove it from the dataset
df = df[df['build_type'] != 'Verschillend']

# Remove rows with bad pictures
ToBeFiltered1 = ['img_dataset/04/0499200002032906-funda.jpg', 'img_dataset/19/1955200000054877-funda.jpg', 'img_dataset/03/0363200000095227-funda.jpg', 'img_dataset/03/0373200000014734-funda.jpg', 'img_dataset/04/0439200000007784-funda.jpg', 'img_dataset/05/0585200001370733-funda.jpg', 'img_dataset/00/0080200000463819-funda.jpg', 'img_dataset/03/0311200000008086-funda.jpg', 'img_dataset/03/0363200000102837-funda.jpg', 'img_dataset/05/0599200100021176-funda.jpg', 'img_dataset/03/0344200000173950-funda.jpg', 'img_dataset/18/1883200000029590-funda.jpg', 'img_dataset/03/0363200012084473-funda.jpg', 'img_dataset/05/0518200000577250-funda.jpg', 'img_dataset/02/0202200000431889-funda.jpg', 'img_dataset/19/1916200000030752-funda.jpg', 'img_dataset/02/0232200000008833-funda.jpg', 'img_dataset/03/0363200000357300-google.jpg', 'img_dataset/18/1895200000018624-funda.jpg', 'img_dataset/02/0236200000015389-funda.jpg', 'img_dataset/07/0794200000074909-google.jpg', 'img_dataset/05/0518200000473313-funda.jpg', 'img_dataset/08/0855200000023936-funda.jpg', 'img_dataset/07/0758200000061431-funda.jpg', 'img_dataset/09/0983200000000991-funda.jpg', 'img_dataset/04/0402200001534400-funda.jpg', 'img_dataset/01/0150200000035293-google.jpg', 'img_dataset/08/0865200000002627-funda.jpg', 'img_dataset/03/0363200000122436-funda.jpg', 'img_dataset/00/0034200000044905-funda.jpg', 'img_dataset/03/0344200000060799-funda.jpg', 'img_dataset/16/1669200000223992-funda.jpg', 'img_dataset/04/0484200000012341-funda.jpg', 'img_dataset/06/0643200002056909-funda.jpg', 'img_dataset/03/0363200000028079-google.jpg', 'img_dataset/09/0983200000005944-funda.jpg', 'img_dataset/03/0373200000022752-funda.jpg', 'img_dataset/02/0275200000102692-funda.jpg', 'img_dataset/01/0153200000403168-funda.jpg', 'img_dataset/16/1676200000478623-funda.jpg', 'img_dataset/04/0406200000069445-funda.jpg', 'img_dataset/00/0010200000107065-funda.jpg', 'img_dataset/05/0512200000020860-funda.jpg', 'img_dataset/05/0599200000323474-funda.jpg', 'img_dataset/00/0034200000057221-funda.jpg', 'img_dataset/03/0363200000302494-funda.jpg', 'img_dataset/03/0353200000008172-funda.jpg', 'img_dataset/03/0363200000223653-funda.jpg', 'img_dataset/05/0579200000003324-funda.jpg', 'img_dataset/16/1640200000078166-funda.jpg', 'img_dataset/05/0599200000382718-funda.jpg', 'img_dataset/04/0479200000004696-funda.jpg', 'img_dataset/09/0995200000795282-funda.jpg', 'img_dataset/01/0160200000252009-funda.jpg', 'img_dataset/00/0034200000056898-funda.jpg', 'img_dataset/05/0512200000104460-google.jpg', 'img_dataset/03/0396200000010315-funda.jpg', 'img_dataset/05/0599200000260925-funda.jpg', 'img_dataset/08/0848200000002437-funda.jpg', 'img_dataset/03/0361200000000721-funda.jpg', 'img_dataset/05/0599200000218706-funda.jpg', 'img_dataset/03/0344200000020337-funda.jpg', 'img_dataset/09/0983200000022324-funda.jpg', 'img_dataset/09/0928200000029978-funda.jpg', 'img_dataset/01/0118200000251470-funda.jpg', 'img_dataset/03/0392200000010646-funda.jpg', 'img_dataset/06/0632200000053444-funda.jpg', 'img_dataset/05/0518200000799233-funda.jpg', 'img_dataset/03/0363200000480310-funda.jpg', 'img_dataset/08/0856200000401324-funda.jpg', 'img_dataset/02/0274200000007934-funda.jpg', 'img_dataset/05/0518200000560249-funda.jpg', 'img_dataset/05/0546200000053110-funda.jpg', 'img_dataset/16/1621200000008145-funda.jpg', 'img_dataset/03/0363200000541853-funda.jpg', 'img_dataset/03/0344200000140853-funda.jpg', 'img_dataset/09/0971200000047182-funda.jpg', 'img_dataset/03/0344200000003048-funda.jpg', 'img_dataset/03/0363200000396611-funda.jpg', 'img_dataset/17/1705200000794783-funda.jpg', 'img_dataset/05/0513200000002660-google.jpg', 'img_dataset/00/0014200010817031-funda.jpg', 'img_dataset/04/0405200001009827-google.jpg', 'img_dataset/03/0392200000007383-funda.jpg', 'img_dataset/05/0575200000056352-funda.jpg', 'img_dataset/03/0363200000295229-funda.jpg', 'img_dataset/03/0344200000124458-google.jpg', 'img_dataset/00/0014200010809240-funda.jpg', 'img_dataset/03/0363200000217961-funda.jpg', 'img_dataset/05/0518200000664147-funda.jpg', 'img_dataset/03/0392200000099054-funda.jpg', 'img_dataset/04/0484200000029110-funda.jpg', 'img_dataset/05/0599200000503988-funda.jpg', 'img_dataset/06/0612200000027700-funda.jpg', 'img_dataset/05/0518200000422499-funda.jpg', 'img_dataset/05/0599200100026286-funda.jpg', 'img_dataset/05/0518200000488664-google.jpg', 'img_dataset/07/0794200000080118-funda.jpg', 'img_dataset/05/0518200000390405-funda.jpg', 'img_dataset/01/0193200000033301-funda.jpg', 'img_dataset/05/0547200000320934-funda.jpg', 'img_dataset/17/1705200000795994-funda.jpg', 'img_dataset/05/0518200000679723-funda.jpg', 'img_dataset/19/1916200000035409-funda.jpg', 'img_dataset/06/0612200000022363-funda.jpg', 'img_dataset/06/0643200002058016-funda.jpg', 'img_dataset/09/0917200000020935-funda.jpg', 'img_dataset/08/0847200000015427-funda.jpg', 'img_dataset/00/0034200000059166-google.jpg', 'img_dataset/03/0363200000536895-funda.jpg', 'img_dataset/05/0599200000192439-funda.jpg', 'img_dataset/03/0363200011943784-funda.jpg', 'img_dataset/05/0503200000042066-funda.jpg', 'img_dataset/02/0267200000006540-funda.jpg', 'img_dataset/04/0463200000002270-funda.jpg', 'img_dataset/03/0310200000693008-funda.jpg', 'img_dataset/01/0114200000371306-funda.jpg', 'img_dataset/03/0392200000098863-funda.jpg', 'img_dataset/05/0599200100021178-funda.jpg', 'img_dataset/02/0222200000657733-funda.jpg', 'img_dataset/04/0405200000565900-google.jpg', 'img_dataset/07/0796200000378739-funda.jpg', 'img_dataset/16/1621200000026387-funda.jpg', 'img_dataset/19/1916200000034047-funda.jpg', 'img_dataset/03/0301200000000312-funda.jpg', 'img_dataset/03/0381200000115691-funda.jpg', 'img_dataset/04/0484200000013761-funda.jpg', 'img_dataset/07/0772200000055829-funda.jpg', 'img_dataset/03/0363200000484669-funda.jpg', 'img_dataset/04/0420200000003686-funda.jpg', 'img_dataset/05/0505200000021778-funda.jpg', 'img_dataset/05/0518200000578428-funda.jpg', 'img_dataset/17/1702200000005124-funda.jpg', 'img_dataset/05/0599200000320881-funda.jpg', 'img_dataset/03/0363200000238279-funda.jpg', 'img_dataset/08/0840200000294721-funda.jpg', 'img_dataset/04/0484200000023370-funda.jpg', 'img_dataset/08/0855200000004784-funda.jpg', 'img_dataset/03/0331200000275597-funda.jpg', 'img_dataset/03/0344200000025644-funda.jpg', 'img_dataset/05/0531200000009108-funda.jpg', 'img_dataset/04/0400200000040820-funda.jpg', 'img_dataset/03/0363200000512153-funda.jpg', 'img_dataset/03/0307200000447323-funda.jpg', 'img_dataset/00/0086200000394817-funda.jpg', 'img_dataset/07/0772200000014062-funda.jpg', 'img_dataset/05/0547200000309874-funda.jpg', 'img_dataset/06/0603200000034456-funda.jpg', 'img_dataset/09/0935200000017292-funda.jpg', 'img_dataset/05/0599200000383336-funda.jpg', 'img_dataset/02/0294200000421532-funda.jpg', 'img_dataset/08/0855200000105242-funda.jpg', 'img_dataset/17/1709200000442652-funda.jpg', 'img_dataset/03/0398200000019217-funda.jpg', 'img_dataset/09/0935200000054573-google.jpg', 'img_dataset/05/0576200000006535-google.jpg', 'img_dataset/03/0344200000162314-funda.jpg', 'img_dataset/02/0241200000062346-funda.jpg', 'img_dataset/05/0599200000234953-funda.jpg', 'img_dataset/03/0363200001969885-funda.jpg', 'img_dataset/05/0599200000752898-funda.jpg', 'img_dataset/04/0479200000020272-funda.jpg', 'img_dataset/03/0356200000008039-funda.jpg', 'img_dataset/06/0637200000193180-funda.jpg', 'img_dataset/03/0307200000563023-funda.jpg', 'img_dataset/03/0327200000267290-funda.jpg', 'img_dataset/04/0402200001581344-funda.jpg', 'img_dataset/03/0398200000022994-funda.jpg', 'img_dataset/18/1883200000002309-funda.jpg', 'img_dataset/02/0200200000077720-google.jpg', 'img_dataset/04/0405200001009440-funda.jpg', 'img_dataset/03/0345200002025714-funda.jpg', 'img_dataset/08/0870200000043551-funda.jpg', 'img_dataset/09/0983200000092624-funda.jpg', 'img_dataset/03/0363200000274814-funda.jpg', 'img_dataset/03/0396200000008489-google.jpg', 'img_dataset/05/0518200000797869-funda.jpg', 'img_dataset/05/0599200000804879-funda.jpg', 'img_dataset/05/0599200000847041-funda.jpg', 'img_dataset/18/1894200000320929-funda.jpg', 'img_dataset/00/0055200000279289-funda.jpg', 'img_dataset/05/0599200000786636-funda.jpg', 'img_dataset/03/0363200012142588-funda.jpg', 'img_dataset/02/0268200000070416-funda.jpg', 'img_dataset/15/1598200000153066-funda.jpg', 'img_dataset/01/0118200000246716-funda.jpg', 'img_dataset/03/0363200000425797-funda.jpg', 'img_dataset/05/0599200000342365-funda.jpg', 'img_dataset/15/1586200000048606-funda.jpg', 'img_dataset/18/1883200000053977-google.jpg', 'img_dataset/04/0406200000071826-funda.jpg', 'img_dataset/01/0150200000032077-funda.jpg', 'img_dataset/06/0642200000023609-funda.jpg', 'img_dataset/03/0363200000209958-funda.jpg', 'img_dataset/03/0355200000759625-funda.jpg', 'img_dataset/05/0576200000010084-funda.jpg', 'img_dataset/05/0518200000432974-funda.jpg', 'img_dataset/17/1700200000035982-funda.jpg', 'img_dataset/00/0014200022201043-funda.jpg']
ToBeFiltered2 = ["/img_dataset/03/0308200000009894-funda.jpg", "/img_dataset/05/0518200000430719-funda.jpg", "/img_dataset/08/0855200000094754-funda.jpg",
                "/img_dataset/00/0074200000374899-funda.jpg", "/img_dataset/04/0484200000023370-funda.jpg", "/img_dataset/02/0297200000010993-funda.jpg",
                "/img_dataset/06/0606200000003641-funda.jpg", "/img_dataset/05/0599200100030186-funda.jpg", "/img_dataset/08/0846200000008069-funda.jpg",
                "/img_dataset/06/0687200000013257-funda.jpg", "/img_dataset/05/0599200000804879-funda.jpg", "/img_dataset/03/0363200012131775-funda.jpg",
                "/img_dataset/03/0307200000464317-funda.jpg", "/img_dataset/03/0344200000107676-funda.jpg", "/img_dataset/03/0363200012110756-funda.jpg",
                "/img_dataset/05/0518200000358818-funda.jpg", "/img_dataset/18/1884200000043106-funda.jpg", "/img_dataset/00/0047200000298237-funda.jpg",
                "/img_dataset/04/0484200000013761-funda.jpg", "/img_dataset/07/0796200000335424-funda.jpg", "/img_dataset/18/1884200000336496-funda.jpg",
                "/img_dataset/07/0757200001448412-funda.jpg", "/img_dataset/05/0518200000727065-funda.jpg", "/img_dataset/19/1916200000035409-funda.jpg",
                "/img_dataset/03/0363200000022176-funda.jpg", "/img_dataset/05/0505200000016786-funda.jpg", "/img_dataset/03/0363200000253056-funda.jpg",
                "/img_dataset/03/0333200000017103-funda.jpg", "/img_dataset/17/1708200000009209-funda.jpg", "/img_dataset/03/0344200000066908-funda.jpg",
                "/img_dataset/01/0148200000010930-funda.jpg","/img_dataset/05/0599200000471860-funda.jpg", "/img_dataset/05/0546200000053909-funda.jpg",
                "/img_dataset/02/0241200000062346-funda.jpg", "/img_dataset/05/0512200000204346-funda.jpg", "/img_dataset/07/0748200000001710-funda.jpg",
                "/img_dataset/05/0546200000019017-funda.jpg", "/img_dataset/05/0599200000193972-funda.jpg", "/img_dataset/05/0518200000734375-funda.jpg",
                "/img_dataset/05/0503200000110371-funda.jpg", "/img_dataset/07/0772200000109343-funda.jpg", "/img_dataset/16/1674200000018465-funda.jpg",
                "/img_dataset/03/0363200000234132-funda.jpg", "/img_dataset/03/0361200000009374-funda.jpg", "/img_dataset/04/0457200000205627-funda.jpg",
                "/img_dataset/05/0553200000006086-funda.jpg", "/img_dataset/15/1581200000120426-funda.jpg", "/img_dataset/03/0361200000013073-funda.jpg",
                "/img_dataset/02/0289200000006993-funda.jpg", "/img_dataset/05/0513200000030238-funda.jpg", "/img_dataset/05/0553200000004775-funda.jpg",
                "/img_dataset/05/0512200000204822-google.jpg", "/img_dataset/05/0599200100021178-funda.jpg", "/img_dataset/00/0050200000359235-funda.jpg",
                "/img_dataset/05/0518200000563090-funda.jpg", "/img_dataset/03/0398200000022994-funda.jpg", "/img_dataset/17/1700200000035982-funda.jpg",
                "/img_dataset/09/0917200000046530-funda.jpg", "/img_dataset/09/0917200000046530-funda.jpg", "/img_dataset/07/0798200000003125-funda.jpg",
                "/img_dataset/05/0531200000003205-funda.jpg", "/img_dataset/01/0114200000381500-funda.jpg", "/img_dataset/01/0171200000015730-funda.jpg",
                "/img_dataset/06/0603200000010227-google.jpg", "/img_dataset/09/0984200000021691-funda.jpg", "/img_dataset/02/0267200000021219-funda.jpg",
                "/img_dataset/05/0505200000029320-funda.jpg", "/img_dataset/05/0518200000682475-funda.jpg", "/img_dataset/05/0599200000262484-funda.jpg",
                "/img_dataset/03/0361200000009654-funda.jpg", "/img_dataset/04/0439200000026107-funda.jpg", "/img_dataset/03/0392200000006359-funda.jpg",
                "/img_dataset/02/0203200001020430-funda.jpg", "/img_dataset/02/0297200000010993-funda.jpg", "/img_dataset/00/0010200000114123-funda.jpg",
                "/img_dataset/03/0344200000036502-funda.jpg", "/img_dataset/03/0362200000080102-funda.jpg", "/img_dataset/09/0983200000081162-google.jpg",
                "/img_dataset/05/0518200000556117-google.jpg", "/img_dataset/03/0363200000391433-funda.jpg", "/img_dataset/07/0796200000412817-funda.jpg",
                "/img_dataset/05/0546200000009249-funda.jpg", "/img_dataset/05/0518200000664147-funda.jpg", "/img_dataset/08/0855200000061228-funda.jpg",
                "/img_dataset/03/0381200000105996-funda.jpg", "/img_dataset/03/0355200000755915-funda.jpg", "/img_dataset/05/0599200000227205-google.jpg",
                "/img_dataset/06/0642200000020104-funda.jpg", "/img_dataset/08/0828200001530531-funda.jpg", "/img_dataset/18/1894200000320929-funda.jpg",
                "/img_dataset/05/0518200000715323-funda.jpg", "/img_dataset/09/0935200000017292-funda.jpg", "/img_dataset/07/0779200000003753-funda.jpg",
                "/img_dataset/02/0202200000836206-funda.jpg", "/img_dataset/01/0114200000346923-funda.jpg", "/img_dataset/05/0599200000321408-funda.jpg",
                "/img_dataset/03/0392200000070983-funda.jpg", "/img_dataset/02/0262200000013473-funda.jpg", "/img_dataset/00/0034200000052269-google.jpg",
                "/img_dataset/03/0361200000020825-funda.jpg", "/img_dataset/05/0599200000761587-funda.jpg", "/img_dataset/05/0599200100026286-funda.jpg",
                "/img_dataset/17/1730200000012590-funda.jpg", "/img_dataset/04/0412200000582839-funda.jpg", "/img_dataset/02/0297200000010993-funda.jpg",
                "/img_dataset/01/0150200000011424-funda.jpg", "/img_dataset/05/0599200000342365-funda.jpg", "/img_dataset/05/0575200000057112-funda.jpg", 
                "/img_dataset/08/0856200000390910-funda.jpg", "/img_dataset/00/0014200010817031-funda.jpg", "/img_dataset/05/0599200100021178-funda.jpg",
                "/img_dataset/05/0599200000514580-funda.jpg", "/img_dataset/05/0518200001760851-funda.jpg", "/img_dataset/08/0828200001572669-funda.jpg",
                "/img_dataset/04/0497200000009291-funda.jpg", "/img_dataset/02/0200200000077720-google.jpg", "/img_dataset/17/1708200000011383-funda.jpg",
                "/img_dataset/03/0340200000347997-funda.jpg", "/img_dataset/03/0344200000152568-funda.jpg", "/img_dataset/07/0718200000024684-funda.jpg",
                "/img_dataset/06/0626200000002226-google.jpg", "/img_dataset/17/1708200000013935-funda.jpg", "/img_dataset/06/0603200000029337-funda.jpg",
                "/img_dataset/05/0599200000752898-funda.jpg", "/img_dataset/16/1652200000002674-funda.jpg", "/img_dataset/18/1842200000007112-funda.jpg", 
                "/img_dataset/19/1916200000005246-funda.jpg", "/img_dataset/05/0595200000001867-funda.jpg", "/img_dataset/03/0363200012081155-funda.jpg"]
ToBeFiltered2 = [path[1:] if path.startswith("/") else path for path in ToBeFiltered2]

ToBeFiltered3 = ['img_dataset/06/0626200000009679-funda.jpg',
                'img_dataset/05/0546200000070346-funda.jpg',
                'img_dataset/05/0518200000370521-funda.jpg',
                'img_dataset/00/0074200000384827-funda.jpg',
                'img_dataset/03/0363200000245030-funda.jpg',
                'img_dataset/05/0599200000518106-funda.jpg',
                'img_dataset/03/0345200002028072-funda.jpg',
                'img_dataset/05/0584200001300079-funda.jpg',
                'img_dataset/02/0228200000028854-funda.jpg',
                'img_dataset/02/0203200000395332-google.jpg',
                'img_dataset/05/0518200000884763-funda.jpg',
                'img_dataset/08/0860200000016907-funda.jpg',
                'img_dataset/03/0363200011073732-funda.jpg,'
                'img_dataset/07/0797200000021976-funda.jpg',
                'img_dataset/03/0358200020615518-funda.jpg',
                'img_dataset/03/0344200000124458-google.jpg',
                'img_dataset/07/0772200000073901-funda.jpg',
                'img_dataset/05/0599200000445891-funda.jpg',
                'img_dataset/05/0542200000669721-funda.jpg',
                'img_dataset/16/1680200000002496-funda.jpg',
                'img_dataset/16/1655200000536112-funda.jpg',
                'img_dataset/19/1916200000037009-funda.jpg',
                'img_dataset/05/0502200000020552-funda.jpg',
                'img_dataset/04/0405200000617364-funda.jpg',
                'img_dataset/03/0394200000044160-funda.jpg',
                'img_dataset/03/0388200000214705-funda.jpg',
                'img_dataset/09/0994200002038094-funda.jpg',
                'img_dataset/00/0014200022207272-funda.jpg',
                'img_dataset/05/0576200000000878-funda.jpg',
                'img_dataset/08/0855200000061228-funda.jpg',
                'img_dataset/05/0599200000179127-funda.jpg',
                'img_dataset/05/0556200000018518-funda.jpg',
                'img_dataset/06/0629200000010036-funda.jpg',
                'img_dataset/05/0512200000025027-funda.jpg',
                'img_dataset/00/0014200010829266-funda.jpg',
                'img_dataset/07/0758200000074375-funda.jpg',
                'img_dataset/00/0080200000393285-funda.jpg',
                'img_dataset/00/0010200000114123-funda.jpg',
                'img_dataset/16/1652200000002674-funda.jpg',
                'img_dataset/07/0798200000003125-funda.jpg',
                'img_dataset/03/0355200000756533-funda.jpg',
                'img_dataset/03/0392200000094746-funda.jpg',
                'img_dataset/03/0344200000120733-funda.jpg',
                'img_dataset/04/0402200001568414-funda.jpg',
                'img_dataset/16/1674200000030839-funda.jpg',
                'img_dataset/07/0770200000005162-funda.jpg',
                'img_dataset/08/0820200000008470-funda.jpg',
                'img_dataset/05/0518200000408233-funda.jpg',
                'img_dataset/05/0513200000009502-funda.jpg',
                'img_dataset/03/0344200000155988-funda.jpg',
                'img_dataset/05/0502200000024597-funda.jpg',
                'img_dataset/03/0344200000069044-funda.jpg',
                'img_dataset/01/0193200000043217-funda.jpg',
                'img_dataset/03/0363200000028277-funda.jpg',
                'img_dataset/19/1916200000038415-funda.jpg',
                'img_dataset/07/0796200000315238-funda.jpg',
                'img_dataset/05/0518200000394166-funda.jpg',
                'img_dataset/00/0034200000016771-funda.jpg',
                'img_dataset/17/1708200000005455-funda.jpg',
                'img_dataset/03/0344200000107676-funda.jpg',
                'img_dataset/05/0599200000388837-funda.jpg',
                'img_dataset/08/0848200000003900-funda.jpg',
                'img_dataset/04/0412200000574267-funda.jpg',
                'img_dataset/07/0718200000002709-funda.jpg',
                'img_dataset/01/0114200000290053-funda.jpg',
                'img_dataset/05/0518200000503514-funda.jpg',
                'img_dataset/03/0363200000238279-funda.jpg',
                'img_dataset/18/1892200000669569-funda.jpg',
                'img_dataset/03/0392200000030839-funda.jpg',
                'img_dataset/06/0637200000249318-funda.jpg',
                'img_dataset/05/0599200000816664-funda.jpg',
                'img_dataset/05/0518200000624596-funda.jpg',
                'img_dataset/05/0518200000356919-funda.jpg',
                'img_dataset/03/0396200000005110-funda.jpg',
                'img_dataset/03/0312200000006450-google.jpg',
                'img_dataset/19/1916200000026168-funda.jpg',
                'img_dataset/07/0717200000120681-funda.jpg',
                'img_dataset/19/1916200000026168-funda.jpg',
                'img_dataset/01/0150200000045805-funda.jpg',
                'img_dataset/05/0599200000203441-funda.jpg',
                'img_dataset/05/0513200000004369-google.jpg',
                'img_dataset/03/0307200000476696-funda.jpg',
                'img_dataset/07/0796200000331259-funda.jpg',
                'img_dataset/19/1916200000014584-funda.jpg',
                'img_dataset/03/0397200000012631-funda.jpg',
                'img_dataset/05/0556200000002909-funda.jpg',
                'img_dataset/04/0473200000011667-funda.jpg',
                'img_dataset/05/0599200000996187-funda.jpg',
                'img_dataset/03/0363200000359130-funda.jpg',
                'img_dataset/02/0228200000040718-google.jpg',
                'img_dataset/03/0307200000490311-funda.jpg',
                'img_dataset/03/0345200002038025-funda.jpg',
                'img_dataset/09/0928200000016930-funda.jpg',
                'img_dataset/06/0637200000262634-funda.jpg',
                'img_dataset/05/0537200000002693-funda.jpg',
                'img_dataset/03/0356200000028300-funda.jpg',
                'img_dataset/07/0772200000110823-funda.jpg',
                'img_dataset/02/0268200000001761-funda.jpg',
                'img_dataset/03/0373200000003500-funda.jpg',
                'img_dataset/05/0599200000197670-funda.jpg',
                'img_dataset/05/0505200000029320-funda.jpg',
                'img_dataset/03/0342200000021018-funda.jpg',
                'img_dataset/07/0796200000373216-funda.jpg',
                'img_dataset/09/0957200000004413-funda.jpg',
                'img_dataset/02/0296200000011667-funda.jpg',
                'img_dataset/05/0503200000020656-funda.jpg',
                'img_dataset/05/0518200000727065-funda.jpg',
                'img_dataset/05/0518200000747314-funda.jpg',
                'img_dataset/07/0756200000001410-funda.jpg',
                'img_dataset/02/0228200000038058-funda.jpg',
                'img_dataset/00/0014200010835104-funda.jpg',
                'img_dataset/03/0344200000193138-funda.jpg',
                'img_dataset/17/1728200000087463-funda.jpg',
                'img_dataset/07/0757200001448412-funda.jpg',
                'img_dataset/01/0193200000097586-funda.jpg',
                'img_dataset/03/0344200000162487-funda.jpg',
                'img_dataset/01/0114200000290455-funda.jpg',
                'img_dataset/04/0439200000032596-funda.jpg',
                'img_dataset/01/0193200000006278-funda.jpg',
                'img_dataset/05/0518200000698190-funda.jpg',
                'img_dataset/03/0307200000461386-funda.jpg',
                'img_dataset/05/0599200000291557-funda.jpg',
                'img_dataset/05/0505200000045486-funda.jpg',
                'img_dataset/16/1699200000013550-funda.jpg',
                'img_dataset/02/0273200000009265-funda.jpg',
                'img_dataset/02/0228200000034328-funda.jpg',
                'img_dataset/03/0361200000025881-funda.jpg',
                'img_dataset/09/0995200000791752-funda.jpg',
                'img_dataset/06/0606200000034366-funda.jpg',
                'img_dataset/06/0606200000026245-funda.jpg',
                'img_dataset/03/0344200000052378-funda.jpg',
                'img_dataset/02/0222200000650621-funda.jpg',
                'img_dataset/08/0882200000012421-funda.jpg',
                'img_dataset/03/0361200000009903-funda.jpg',
                'img_dataset/05/0597200000272412-funda.jpg',
                'img_dataset/03/0363200000023540-funda.jpg',
                'img_dataset/03/0363200000069149-funda.jpg',
                'img_dataset/07/0796200000319596-google.jpg',
                'img_dataset/03/0363200000022176-funda.jpg',
                'img_dataset/18/1883200000051573-funda.jpg',
                'img_dataset/05/0599200000822594-funda.jpg',
                'img_dataset/08/0899200000278456-funda.jpg',
                'img_dataset/03/0363200000420927-funda.jpg',
                'img_dataset/05/0599200100030186-funda.jpg',
                'img_dataset/03/0344200000036502-funda.jpg',
                'img_dataset/09/0917200000046530-funda.jpg',
                'img_dataset/03/0373200000012089-funda.jpg',
                'img_dataset/00/0014200010876958-funda.jpg',
                'img_dataset/01/0119200000013124-funda.jpg',
                'img_dataset/05/0547200000307944-funda.jpg',
                'img_dataset/03/0344200000116028-funda.jpg',
                'img_dataset/08/0855200000076557-funda.jpg',
                'img_dataset/03/0398200000006242-funda.jpg',
                'img_dataset/05/0599200100045415-funda.jpg',
                'img_dataset/05/0599200000503460-funda.jpg',
                'img_dataset/00/0034200000064013-funda.jpg',
                'img_dataset/18/1884200000035232-google.jpg',
                'img_dataset/08/0899200000274182-funda.jpg',
                'img_dataset/07/0796200000379634-funda.jpg',
                'img_dataset/03/0344200000103360-funda.jpg',
                'img_dataset/00/0014200010838855-funda.jpg',
                'img_dataset/15/1525200000010472-funda.jpg',
                'img_dataset/06/0603200000019912-funda.jpg',
                'img_dataset/09/0995200000789904-funda.jpg',
                'img_dataset/16/1659200000007138-funda.jpg',
                'img_dataset/04/0402200001551666-funda.jpg',
                'img_dataset/05/0546200000053909-funda.jpg',
                'img_dataset/01/0150200000033017-funda.jpg',
                'img_dataset/05/0576200000004039-funda.jpg',
                'img_dataset/05/0512200000204822-google.jpg',
                'img_dataset/05/0546200000064381-funda.jpg',
                'img_dataset/01/0114200000322569-funda.jpg',
                'img_dataset/02/0268200000038871-funda.jpg',
                'img_dataset/01/0173200000171050-funda.jpg',
                'img_dataset/03/0344200000136935-funda.jpg',
                'img_dataset/05/0545200000366203-google.jpg']

ToBeFiltered = list(set(ToBeFiltered1 + ToBeFiltered2 + ToBeFiltered3))
print(len(ToBeFiltered))

def pick_first_url(row):
    for col in [f"{c}_split" for c in link_cols]:
        val = row[col]
        if pd.notna(val) and val != '':
            return val
    return ''

df['frontview_url'] = df.apply(pick_first_url, axis=1)
df = df[df['frontview_url'] != '']

df = df[df['frontview_url'].isin(ToBeFiltered) == False]

# Ensure 'opp_pand' and 'oppervlakte' are numeric before division
df['procent_ingenomen'] = pd.to_numeric(df['opp_pand'], errors='coerce') / pd.to_numeric(df['oppervlakte'], errors='coerce')

df['huisnr_bag_letter'] = df['huisnr_bag_letter'].notna().astype(int)
df['huisnr_bag_toevoeging'] = df['huisnr_bag_toevoeging'].notna().astype(int)

df['is_monument'] = df['is_monument'].fillna(0).astype(int)
df['is_protected'] = df['is_protected'].fillna(0).astype(int)

df = df.drop(columns=['bag_nummeraanduidingid', 'frontview_exists', 'random_rank', 'num_funda_images',
                      'frontview_funda_url', 'frontview_google_url', 'frontview_funda_in_business_url', 
                      'frontview_funda_url_split', 'frontview_google_url_split', 'frontview_funda_in_business_url_split',
                      'special_house_type', 'source_data_result_id', 'source_data_timestamp',
                      'straatnaam', 'postcode', 'plaatsnaam', 'oppervlakte',
                      ])

merge_map = {
    'Bovenwoning': 'Bovenwoning/Benedenwoning/Maisonette',
    'Benedenwoning': 'Bovenwoning/Benedenwoning/Maisonette',
    'Maisonnette': 'Bovenwoning/Benedenwoning/Maisonette',
    'Corridorflat': 'Corridorflat/Galerijflat',
    'Galerijflat': 'Corridorflat/Galerijflat',
    'Hoekwoning': 'Hoekwoning/Eindwoning',
    'Eindwoning': 'Hoekwoning/Eindwoning',
    'Portiekflat': 'Portiekflat/Portiekwoning',
    'Portiekwoning': 'Portiekflat/Portiekwoning'

    # etc.
}

df['woningtype'] = df['woningtype'].map(merge_map).fillna(df['woningtype'])

# Full preprocessed dataset with URLS, can be loaded into pipeline.
df.to_csv(
    file_path + "Full_preprocessed_detailed_house.csv",
    index=False,
    encoding='utf-8',
)

466


In [5]:
def compute_orientation(polygon):
    if polygon.is_empty or not polygon.is_valid:
        return np.nan

    # Get minimum rotated rectangle
    mrr = polygon.minimum_rotated_rectangle
    coords = list(mrr.exterior.coords)

    # Find the longest edge
    max_length = 0
    angle = 0

    for i in range(len(coords) - 1):
        p1 = coords[i]
        p2 = coords[i + 1]

        dx = p2[0] - p1[0]
        dy = p2[1] - p1[1]

        length = np.hypot(dx, dy)
        if length > max_length:
            max_length = length
            angle = np.degrees(np.arctan2(dy, dx))

    # Normalize angle to 0–180
    return angle % 180

def compute_elongation(polygon):
    if polygon.is_empty or not polygon.is_valid:
        return np.nan

    # Minimum rotated rectangle (oriented bounding box)
    min_rect = polygon.minimum_rotated_rectangle

    # Get corner points of the box
    coords = list(min_rect.exterior.coords)

    # Compute distances between the 4 sides
    edge_lengths = [np.linalg.norm(np.subtract(coords[i], coords[i + 1])) for i in range(4)]

    width = min(edge_lengths)
    height = max(edge_lengths)

    if height == 0:  # Prevent divide-by-zero
        return np.nan

    return width / height

def rasterize_polygon(geom, size=224):
    bounds = geom.bounds
    geom = translate(geom, xoff=-bounds[0], yoff=-bounds[1])
    scale_x = size / (bounds[2] - bounds[0] + 1e-8)
    scale_y = size / (bounds[3] - bounds[1] + 1e-8)
    geom = scale(geom, xfact=scale_x, yfact=scale_y, origin=(0, 0))

    img = Image.new("L", (size, size), 0)
    draw = ImageDraw.Draw(img)
    coords = [(x, size - y) for x, y in geom.exterior.coords]
    draw.polygon(coords, outline=1, fill=1)
    return np.array(img)

In [6]:
df['geometry_wkt'] = df['geometry'].apply(wkt.loads)

df['centroid_x'] = df['geometry_wkt'].apply(lambda geom: geom.centroid.x)
df['centroid_y'] = df['geometry_wkt'].apply(lambda geom: geom.centroid.y)

df['area'] = df['geometry_wkt'].apply(lambda geom: geom.area)

# perimeter = Sum of the lengths of all edges forming the boundary of a polygon
df['perimeter'] = df['geometry_wkt'].apply(lambda g: g.length)

# Gives 1 for a perfect circle (most compact shape)
# Gets closer to 0 for long, skinny, jagged shapes
df['compactness'] = (
    4 * np.pi * df['area'] / (df['perimeter'] ** 2)
)

df['num_vertices'] = df['geometry_wkt'].apply(lambda g: len(g.exterior.coords))

df['elongation'] = df['geometry_wkt'].apply(compute_elongation)

df['orientation_deg'] = df['geometry_wkt'].apply(compute_orientation)

df['num_vertices_log'] = np.log1p(df['num_vertices'])

df.to_csv(
    file_path + "Full_preprocessed_detailed_house.csv",
    index=False,
    encoding='utf-8',
)

print(df.dtypes)

df["mask"] = df["geometry_wkt"].apply(lambda g: rasterize_polygon(g, size=224))

woningtype                       object
huisnr                   string[python]
huisnr_bag_letter                 int64
huisnr_bag_toevoeging             int64
opp_pand                 string[python]
build_year               string[python]
build_type               string[python]
is_monument                       int64
is_protected                      int64
geometry                 string[python]
bag_id                   string[python]
frontview_url                    object
procent_ingenomen               Float64
geometry_wkt                     object
centroid_x                      float64
centroid_y                      float64
area                            float64
perimeter                       float64
compactness                     float64
num_vertices                      int64
elongation                      float64
orientation_deg                 float64
num_vertices_log                float64
dtype: object


In [7]:
# Normalizing and feature engineering are applied to the dataset separately for training, validation, and testing.
# This is to ensure that the model does not learn from the validation and test sets during training.

# adjust random_state for reproducibility
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

numeric_cols = ['opp_pand', 'build_year', 'huisnr', 'procent_ingenomen']

scaler = StandardScaler()
train_df[numeric_cols] = scaler.fit_transform(train_df[numeric_cols])
val_df[numeric_cols] = scaler.transform(val_df[numeric_cols])
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])


for col in ['centroid_x', 'centroid_y']:
    min_val = train_df[col].min()
    max_val = train_df[col].max()
    for df in [train_df, val_df, test_df]:
        df[col] = (df[col] - min_val) / (max_val - min_val + 1e-8)

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoder.fit(train_df[['build_type']])
feature_names = [name.replace(' ', '_') for name in encoder.get_feature_names_out(['build_type'])]

for df in [train_df, val_df, test_df]:
    encoded_array = encoder.transform(df[['build_type']])
    encoded_df = pd.DataFrame(encoded_array, columns=feature_names, index=df.index)
    df.drop(columns='build_type', inplace=True)
    df[encoded_df.columns] = encoded_df

In [7]:
features = [
    # BAG
    'area',
    'centroid_x',                               # 0 - 1
    'centroid_y',                               # 0 - 1
    'perimeter',                                # 0 - 1
    'compactness',                              # 0 - 1
    'elongation',                               # 0 - 1
    'orientation_deg',                          # 0 - 1
    'num_vertices_log',                         # 0 - 7 (can be inf)
]

target = 'woningtype'

In [9]:
df_sample = pd.read_csv(file_path + "detailed_woning_type_sample.csv")
df = pd.read_csv(file_path + "bag_image_summary.csv", dtype="string")
# Ensure both columns are string for matching
df_sample["bag_nummeraanduidingid"] = df_sample["bag_nummeraanduidingid"].astype(str)
df["bag_id"] = df["bag_id"].astype(str)

# Keep only rows in df whose bag_id is NOT in df_sample
df = df[~df["bag_id"].isin(df_sample_with_urls["bag_nummeraanduidingid"])]

# If you want to add the file path to the URLs, set this to True
add_file_path_to_urls = False

# Currently a funda sourced Url goes from: 
# frontview/0797/2000/0002/3888/0797200000023888.jpg
# to: img_dataset/07/079720000002-funda.jpg
def extract_path(url, source):
    if pd.isna(url) or url == '' or url is None:
        return ''
    id = url.rstrip('/').split('/')[-1]
    id, *_ = id.split('.')
    m = re.match(r'(\d{2})', id)
    first_two_digits = m.group(1) if m else ''
    return f"img_dataset/{first_two_digits}/{id}-{source}.jpg"

link_cols = ['frontview_funda_url', 'frontview_google_url', 'frontview_funda_in_business_url']
link_sources = ['funda', 'google', 'funda-in-business'] # Sources are in file name, so need to be added to filename for correct name

for col, source in zip(link_cols, link_sources):
    df[f'{col}_split'] = df[col].map(lambda url: extract_path(url, source))

# If you want to add the file path to the URLs, set add_file_path_to_urls to True
if add_file_path_to_urls:
    df[[f'{col}_split' for col in link_cols]] = df[[f'{col}_split' for col in link_cols]].map(lambda x: file_path + x if x else '')
    add_file_path_to_urls = False

def pick_first_url(row):
    for col in [f"{c}_split" for c in link_cols]:
        val = row[col]
        if pd.notna(val) and val != '':
            return val
    return ''

# Filter out rows where funda url is empty
df = df[df['frontview_funda_url'].notna() & (df['frontview_funda_url'] != '')]

df['frontview_url'] = df.apply(pick_first_url, axis=1)
df = df[df['frontview_url'] != '']

df = df.drop(columns=['frontview_exists', 'num_funda_images',
                      'frontview_funda_url', 'frontview_google_url', 'frontview_funda_in_business_url', 
                      'frontview_funda_url_split', 'frontview_google_url_split', 'frontview_funda_in_business_url_split',
                      ])

df.to_csv(
    file_path + "Preprocessed_pretraining_set.csv",
    index=False,
    encoding='utf-8',
)

In [ ]:
train_df.to_csv(os.path.join(file_path, "train_df.csv"), index=False)
val_df.to_csv(os.path.join(file_path, "val_df.csv"), index=False)
test_df.to_csv(os.path.join(file_path, "test_df.csv"), index=False)

print("Saved train_df.csv, val_df.csv and test_df.csv to:", file_path)

Saved train_df.csv, val_df.csv and test_df.csv to: ../../Data/


In [9]:
len(train_df)

5648

In [10]:
for col in df.columns:
    print(col)

woningtype
huisnr
huisnr_bag_letter
huisnr_bag_toevoeging
opp_pand
oppervlakte
build_year
build_type
is_monument
is_protected
frontview_url
procent_ingenomen
centroid_x
centroid_y
area
perimeter
compactness
elongation
orientation_deg
num_vertices_log
mask


In [12]:
# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Dataset
class MultiModalDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.features = features
        self.transform = transform
        self.counter = 0

        self.masks = [torch.tensor(m, dtype=torch.float32).unsqueeze(0) for m in df["mask"]]
        self.tabular = torch.tensor(df[features].values, dtype=torch.float32)
        self.labels = torch.tensor(df["woningtype_encoded"].values, dtype=torch.long)
        
        self._remove_missing()


    def _remove_missing(self):
        to_drop = []
        for idx, row in self.df.iterrows():
            try:
                Image.open(row['frontview_url'])
            except Exception as e:
                to_drop.append(idx)
                self.counter += 1
                print(f'Dropped {self.counter}th row due to image load error: {e}')
        
        # Drop all invalid rows at once
        self.df = self.df.drop(index=to_drop).reset_index(drop=True)


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Get mask and tabular
        mask = self.masks[idx]
        tabular = self.tabular[idx]
        label = self.labels[idx]

        # Load image
        try:
            img = Image.open(row['frontview_url']).convert('RGB')
            if self.transform:
                img = self.transform(img)
            else:
                img = torch.tensor(np.array(img), dtype=torch.float32).permute(2, 0, 1) / 255.0  # normalize manually
        except Exception as e:
            # Handle failed image read by recursively calling next index
            print(e)
            return self.__getitem__((idx + 1) % len(self))

        return img, mask, tabular, label

# Model
class CNNWithTabular(nn.Module):
    def __init__(self, image_out_dim, tabular_dim, output_dim):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(3),
            nn.Conv2d(16, 64, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(3),
            nn.Dropout(0.2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(),
            nn.MaxPool2d(3),
            nn.Flatten(),
            nn.Dropout(0.2),
            nn.Linear(128 * 8 * 8, image_out_dim), nn.ReLU()
        )

        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_dim, 512), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 128), nn.ReLU(),
        )

        self.tab_scale = nn.Parameter(torch.ones(1) * 1.5)  

        self.final = nn.Sequential(
            nn.Linear(image_out_dim + 128, 512), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256), nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, output_dim)
        )

    def forward(self, img, tab):
        img_out = self.cnn(img)
        tab_out = self.tabular_net(tab)
        tab_out = tab_out * self.tab_scale
        x = torch.cat([img_out, tab_out], dim=1)
        return self.final(x)

Using device: cuda


In [13]:
# Encode labels
le = LabelEncoder()
train_df["woningtype_encoded"] = le.fit_transform(train_df[target])
val_df["woningtype_encoded"] = le.transform(val_df[target])
test_df["woningtype_encoded"] = le.transform(test_df[target])

# Datasets and loaders
train_ds = MultiModalDataset(train_df)
val_ds = MultiModalDataset(val_df)
test_ds = MultiModalDataset(test_df)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)
test_loader = DataLoader(test_ds, batch_size=32)

# Model, loss, optimizer
model = CNNWithTabular(image_out_dim=128, tabular_dim=len(features), output_dim=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    weight_decay=1e-5
)

# Training loop
loss_history = []

for epoch in range(16):
    model.train()
    total_loss = 0
    for x_img, x_mask, x_tab, y_batch in train_loader:
        x_img, x_mask, x_tab, y_batch = x_img.to(device), x_mask.to(device), x_tab.to(device), y_batch.to(device)
        optimizer.zero_grad()
        out = model(x_img, x_tab)
        loss = criterion(out, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    loss_history.append(avg_loss)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}")

# Evaluation
all_preds, all_true, all_probs = [], [], []

model.eval()
with torch.no_grad():
    for x_img, x_tab, y_batch in test_loader:
        x_img, x_tab, y_batch = x_img.to(device), x_tab.to(device), y_batch.to(device)
        out = model(x_img, x_tab)
        probs = torch.softmax(out, dim=1)
        preds = probs.argmax(dim=1)
        
        all_preds.extend(preds.cpu().tolist())
        all_true.extend(y_batch.cpu().tolist())
        all_probs.extend(probs.cpu().numpy())

print("\n--- Evaluation Metrics ---")
accuracy = accuracy_score(all_true, all_preds)
precision = precision_score(all_true, all_preds, average='macro', zero_division=0)
recall = recall_score(all_true, all_preds, average='macro', zero_division=0)
f1 = f1_score(all_true, all_preds, average='macro', zero_division=0)
kappa = cohen_kappa_score(all_true, all_preds)
logloss = log_loss(all_true, all_probs)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision (macro): {precision:.4f}")
print(f"Recall (macro): {recall:.4f}")
print(f"F1 Score (macro): {f1:.4f}")
print(f"Cohen's Kappa: {kappa:.4f}")
print(f"Log Loss: {logloss:.4f}")

# --- Classification Report ---
print("\n--- Classification Report ---") 
print(classification_report(all_true, all_preds, target_names=le.classes_, zero_division=0))

Dropped 1th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0938200000025776-funda.jpg'
Dropped 2th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0984200000013978-funda.jpg'
Dropped 3th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0917200000025392-funda.jpg'
Dropped 4th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0957200000019190-funda.jpg'
Dropped 5th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0988200000035883-funda.jpg'
Dropped 6th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0988200000049049-funda.jpg'
Dropped 7th row due to image load error: [Errno 2] No such file or directory: '../../Data/img_dataset/09/0907200000015447-funda.jpg'
Dropped 8th row due to image load error: [Errno 2] No such file or di

RuntimeError: stack expects each tensor to be equal size, but got [3, 960, 1440] at entry 0 and [3, 640, 640] at entry 2

In [ ]:
print("\n--- Confusion Matrix ---")
cm = confusion_matrix(all_true, all_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()  

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(range(1, len(loss_history) + 1), loss_history, marker='o')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.show()